# Route Montag 
### (Vormittag) 
<br>
Since the author recommends to visit surrounding cities in the afternoon (which are not on my map), I decided to limit my analysis to the late morning. <br>
But of course, those would suffice as interesting sources as well. 

In [10]:
from os import path
xml_path = path.relpath("../Data/pages_8_47.xml")

from bs4 import BeautifulSoup   
with open(xml_path, encoding='utf8') as file:   
    contents = file.read()
    soup = BeautifulSoup(contents, 'xml')

In [15]:
import re

start_tag = 'hi'   
start_string = 'Montag.'    
end_tag = 'hi'     
end_string = 'Dinstag.'  
place_tag = 'placeName'     
should_start_parsing = False   

In [16]:
Montag = []        #here, the whole text will be placed - in case we need it later on
Places = []     #here, the places for our route will be stored
for line in soup.find_all('l'):
    if line.string == start_string and line.string.parent.name == start_tag:
        should_start_parsing = True  

    if should_start_parsing:
        text = line.get_text()
        Montag.append(text)
        for child in line.descendants:
            if child.name == place_tag:
                for text in child.strings:
                    Places.append(text)          

    if line.string == end_string and line.string.parent.name == end_tag:
        should_start_parsing = False  
        break

In [17]:
while(" " in Places):
    Places.remove(" ")
print(Places) 

clean_places = []
for e in Places:
    if clean_places and clean_places[-1].endswith('='):
        clean_places[-1] = clean_places[-1][:-1] + e
    elif clean_places and (clean_places[-1].endswith('des ') or clean_places[-1].endswith('des')):
        clean_places[-1] = clean_places[-1] + ' ' + e
    elif clean_places and (clean_places[-1].endswith('der ') or clean_places[-1].endswith('der')): 
        clean_places[-1] = clean_places[-1] + ' ' + e     
    elif clean_places and clean_places[-1].endswith('den '): 
        clean_places[-1] = clean_places[-1] + ' ' + e 
    elif clean_places and (clean_places[-1].endswith('Fürsten ') or clean_places[-1].endswith('.')):
        clean_places[-1] = clean_places[-1] + ' ' + e         
    else:
        clean_places.append(e)


['kaiserlichen', 'Zeughauses', 'Renngasse Nr. 140', 'Renngasse', 'hohe Brücke', 'tiefen Graben', 'Schwert=', 'gasse', 'Maria Stiegen', 'Salzgries', 'Maria am Gestade', 'Stoß im Himmel', 'Wipplingerstraße', 'Rathhaus', 'Hofkanzlei', 'Quergasse', 'Judenplatz', 'den Hof.', 'Kirche zu den ', '7 Chören der Engel', 'bürgerliche Zeughaus', 'Bognergasse', 'Seitzerhof', 'Spenglergasse', 'Milchgäßchen', 'Petersplatz', 'Peterskirche', 'Polizei=', 'Oberdirektion', 'Feuerwache', 'Goldschmiedgasse', 'Stephansplatz', 'Gerolds', ' Buchhandlung', 'Stephansdom', 'Stephansplatze', 'Bischofgasse', 'Wollzeile', 'Schranders ', 'Kleidermagazin', 'Brief=', 'Post', 'Schaumburgs eleganter Bücher=', 'laden', 'Essiggäßchen', 'Bäckerstraße', 'Universitätsplatz', 'Universitäts=Gebäude', 'Sternwarte', 'Konvikt', 'Uni=', 'versitätshaus', 'Schulgasse', 'Dominkanerplatz', 'Universitätsbibllothek', 'Fahrpost', 'grie=', 'chische Kirche', 'Hauptmauth', 'Hôtel zur Stadt London', 'Klosternenburg', 'Sommer=', 'palais', 'Gart

In [18]:
print(clean_places)
print(len(clean_places))

['kaiserlichen', 'Zeughauses', 'Renngasse Nr. 140', 'Renngasse', 'hohe Brücke', 'tiefen Graben', 'Schwertgasse', 'Maria Stiegen', 'Salzgries', 'Maria am Gestade', 'Stoß im Himmel', 'Wipplingerstraße', 'Rathhaus', 'Hofkanzlei', 'Quergasse', 'Judenplatz', 'den Hof. Kirche zu den  7 Chören der Engel', 'bürgerliche Zeughaus', 'Bognergasse', 'Seitzerhof', 'Spenglergasse', 'Milchgäßchen', 'Petersplatz', 'Peterskirche', 'PolizeiOberdirektion', 'Feuerwache', 'Goldschmiedgasse', 'Stephansplatz', 'Gerolds', ' Buchhandlung', 'Stephansdom', 'Stephansplatze', 'Bischofgasse', 'Wollzeile', 'Schranders ', 'Kleidermagazin', 'BriefPost', 'Schaumburgs eleganter Bücherladen', 'Essiggäßchen', 'Bäckerstraße', 'Universitätsplatz', 'Universitäts=Gebäude', 'Sternwarte', 'Konvikt', 'Universitätshaus', 'Schulgasse', 'Dominkanerplatz', 'Universitätsbibllothek', 'Fahrpost', 'griechische Kirche', 'Hauptmauth', 'Hôtel zur Stadt London', 'Klosternenburg', 'Sommerpalais', 'Garten des  Fürsten  Liechtenstein', 'Rossau'

In [6]:
#manual refinement: 
import functools

clean_places[0:2] = [functools.reduce(lambda x, y: x + ' ' + y, clean_places[0:2])]

In [7]:
#again: compare words to compile a list with only one version of varying place_names in the list

import difflib as dl

Similiar_words = []
for word in clean_places:
    close_words = dl.get_close_matches(word, clean_places, cutoff= 0.9)     #find matches of similar written places
    Similiar_words.append(close_words)                                      #gather them in a list

place_list = []
for sublist in Similiar_words:     #only the first word or version of every similar-places-combination gets chosen as 'standard'
    sublist = sorted(sublist)
    place = sublist[0]
    place_list.append(place)  

place_list_text= open('../Data/monday_place_list_text.txt','w', encoding='utf8')      #save the new list of places in a file
for item in place_list:
	place_list_text.write(item+"\n")
place_list_text.close()      

Let's start building our coordinate-table! <br>
As before, We will start with the df from Wien Geschichte Wiki, add the coordinates from the Gazetteer and finally compare it to the manually built dataframe! 

In [8]:
import pandas as pd
csv_path = path.relpath("../Data/KULTURWIKIOGD.csv")    #import the downloaded dataset
usecolumns = ['OBJECTID', 'SHAPE', 'ADRESSE', 'SEITENNAME']    #since the dataset is pretty big, I decided to use only some of the columns for efficieny
coordinates = pd.read_csv(csv_path,  index_col="OBJECTID", usecols=usecolumns)

from thefuzz import process, fuzz 

coordinates['MATCH_SCORE'] = coordinates['SEITENNAME'].apply(lambda x: process.extractOne(x, place_list, scorer=fuzz.ratio)[1]) 
# create a new column called MATCH_SCORE, where the resulting scores of the comparison between our place_list will be stored. 

coordinates = coordinates.sort_values('MATCH_SCORE', ascending=False)
coordinates_choice = coordinates.drop(coordinates[coordinates['MATCH_SCORE'] <= 84].index) #I used 84 again as threshold, since it fit pretty well

coordinates_choice['NEUE_NAMEN'] = coordinates_choice['SEITENNAME'].apply(lambda x: process.extractOne(x, place_list, scorer=fuzz.ratio)[0]) 
#  use uniform names for our places 

coordinates_choice = coordinates_choice.drop_duplicates(subset=['NEUE_NAMEN']) 
place_coord  = coordinates_choice.drop(columns = ['SEITENNAME', 'MATCH_SCORE']) #drop unnceccessary columns

place_coord = place_coord.set_index('NEUE_NAMEN')  #set new index
place_coord = place_coord.reindex(index=place_list) #re-index again: corresponding records get ordered according to the place list
place_coord = place_coord.reset_index() #reset the index
place_coord.reset_index(inplace=True)
place_coord = place_coord.drop(columns=['index'])
place_coord.to_csv('../Data/Montag_coord.csv', sep=',', index=False, encoding='utf-8') #save our new Dataframe of places

In [9]:
place_coord

,NEUE_NAMEN,SHAPE,ADRESSE
0,kaiserlichen Zeughauses,NaN,NaN
1,Renngasse Nr. 140,NaN,NaN
2,Renngasse,POINT (16.367211551082715 48.21277258486223),"01., Renngasse 12"
3,hohe Brücke,POINT (16.368607210105097 48.21273703123713),"01., Hohe Brücke 120"
4,tiefen Graben,POINT (16.36762957026213 48.212022628343945),"01., Tiefer Graben 12"
...,...,...,...
321,Halterthal,NaN,NaN
322,Hütteldorf,NaN,NaN
323,Tulbingkogel,NaN,NaN
324,Mauerbach,NaN,NaN


Again, let's also try the Gazetteer: 